# Zadania do wykonania

1. Porównaj zapisywanie i odczytywanie kolekcji (100, 10000, 100 000 elementów) za pomocą trzech technik: modułu `pickle`, `parquet` i `xlsx`.

2. Zbadaj przyspieszenie związane z zastosowaniem pamięci podręcznej na wybranych przykładzie funkcji rekurencyjnej (np. ciągu _Fibonacciego_.

3. Napisz program tworzący bazę danych z interfejsem konsolowym. Wymagane są następujące operacje, dodanie wiersza, usunięcie wiersza, zmiana pola wiersza, wyświetlenie opcji. Menu można zorganizować jako odczytywanie parametrów zwróconych przez funkcję `input` lub z pomocą komend (łatwiejszy sposób).

In [10]:
# zadanie 1


import openpyxl
import pandas
import time
import json 
from openpyxl.reader.excel import load_workbook
from fastparquet import ParquetFile
from tempfile import mktemp
from pickle import dump, load
from os import unlink



pickleR = 0
pickleW = 0

fileForPickle = mktemp();
fileForExcel = mktemp(suffix='.xlsx');
fileForParquet = mktemp();

with open('books.json') as json_file:
    dataHundret = json.load(json_file);
with open('books2.json') as json_file:
    dataThousand = json.load(json_file);




def createPickle(data):
    start = time.time()
    
    with open(fileForPickle, 'wb') as f:
        dump(data, f)
        f.flush()    
    
    end = time.time()
    return end - start
        
def readPickle():
    start = time.time()
    
    with open(fileForPickle, 'rb') as f:
        temp_data = load(f)
    
    end = time.time()
    return end - start

def createXlsx(data):
    start = time.time()

    df = pandas.DataFrame(data)
    df.to_excel(fileForExcel)
    
    end = time.time()
    return end - start

def readXlsx():
    start = time.time()
    
    wb = load_workbook(fileForExcel)
    pandas.read_excel(fileForExcel, sheet_name=None)
    
    end = time.time()
    return end - start

def createParquet(data):
    start = time.time()
    
    df = pandas.DataFrame.from_dict(data)
    df.to_parquet(fileForParquet, compression='GZIP')
    
    end = time.time()
    return end - start    

def readParquet():
    start = time.time()
    
    pf = ParquetFile(fileForParquet)
    df = pf.to_pandas()
    
    end = time.time()
    return end - start         
    

# Pickle
try:
    print('Modul Pickle: ')
    print('Zapisywanie: ')
    print(f'\t 100 wierszy: {createPickle(dataHundret)} [s]')
    print(f'\t 100 wierszy: {readPickle()} [s]')
    print('Odczytywanie: ')
    print(f'\t 1000 wierszy: {createPickle(dataThousand)} [s]')
    print(f'\t 1000 wierszy: {readPickle()} [s]')
finally:
    unlink(fileForPickle)
# Xlsx
try:
    print('Modul Xlsx: ')
    print('Zapisywanie: ')
    print(f'\t 100 wierszy: {createXlsx(dataHundret)} [s]')
    print(f'\t 100 wierszy: {readXlsx()} [s]')
    print('Odczytywanie: ')
    print(f'\t 1000 wierszy: {createXlsx(dataThousand)} [s]')
    print(f'\t 1000 wierszy: {readXlsx()} [s]')
finally:
    unlink(fileForExcel)
# Parquet
try:
    print('\nModul Parquet: ')
    print('Zapisywanie: ')
    print(f'\t 100 wierszy: {createParquet(dataHundret)} [s]')
    print(f'\t 100 wierszy: {readParquet()} [s]')
    print('Odczytywanie: ')
    print(f'\t 1000 wierszy: {createParquet(dataThousand)} [s]')
    print(f'\t 1000 wierszy: {readParquet()} [s]')
finally:
    unlink(fileForParquet)

Modul Pickle: 
Zapisywanie: 
	 100 wierszy: 0.002035379409790039 [s]
	 100 wierszy: 0.00024390220642089844 [s]
Odczytywanie: 
	 1000 wierszy: 0.0022573471069335938 [s]
	 1000 wierszy: 0.0012691020965576172 [s]
Modul Xlsx: 
Zapisywanie: 
	 100 wierszy: 0.06106734275817871 [s]
	 100 wierszy: 0.11815762519836426 [s]
Odczytywanie: 
	 1000 wierszy: 0.4812753200531006 [s]
	 1000 wierszy: 0.6297106742858887 [s]

Modul Parquet: 
Zapisywanie: 
	 100 wierszy: 0.016669750213623047 [s]
	 100 wierszy: 0.0048711299896240234 [s]
Odczytywanie: 
	 1000 wierszy: 0.0239713191986084 [s]
	 1000 wierszy: 0.005168914794921875 [s]


In [4]:
import sys
!{sys.executable} -m pip install fastparquet

     |████████████████████████████████| 28.6 MB 87 kB/s  eta 0:00:012   |▉                               | 706 kB 1.6 MB/s eta 0:00:18     |███████████▊                    | 10.5 MB 6.1 MB/s eta 0:00:03     |█████████████▊                  | 12.2 MB 6.1 MB/s eta 0:00:03     |███████████████████             | 16.9 MB 5.3 MB/s eta 0:00:03███████████████████████▌      | 22.8 MB 5.3 MB/s eta 0:00:02     |█████████████████████████████▎  | 26.1 MB 6.0 MB/s eta 0:00:01
     |████████████████████████████████| 59 kB 2.7 MB/s eta 0:00:01
  Created wheel for fastparquet: filename=fastparquet-0.4.1-cp38-cp38-linux_x86_64.whl size=7158673 sha256=2a53165b24904a381b65507a5399ac2e0c9124671f688972e521a8ee50e5d5f9
  Stored in directory: /home/jovyan/.cache/pip/wheels/bd/9a/95/3fbcf7e8b3f379ca499c994b956208d91778b1b1bb39c9208d
  Created wheel for thrift: filename=thrift-0.13.0-cp38-cp38-linux_x86_64.whl size=413815 sha256=e3c09247241651008b92cdb1795cc435aaf5d75c8ee75548a4bfe7e1b19bea72
  Stored in direct

In [2]:
import sys
!{sys.executable} -m pip install openpyxl

     |████████████████████████████████| 242 kB 1.6 MB/s eta 0:00:01
  Created wheel for et-xmlfile: filename=et_xmlfile-1.0.1-py3-none-any.whl size=8917 sha256=bdd2d825c0848383eef6d8329a1a3c048124147f818e619fdaa4e1f4b247f4fa
  Stored in directory: /home/jovyan/.cache/pip/wheels/6e/df/38/abda47b884e3e25f9f9b6430e5ce44c47670758a50c0c51759
Successfully built et-xmlfile


In [90]:
# zadanie 2

import time
from functools import lru_cache, cached_property


# Recursive algorithm for fibbonacci
def fib(n):
    if n <= 1:
        return n
    else:
        return(fib(n-1) + fib(n-2))
        
        
@lru_cache(maxsize=100)
def func_lru_cached(a: int):
    if a <= 1:
        return a
    else:
        return(func_lru_cached(a-1) + func_lru_cached(a-2))        
       

def exec(number):
    start = time.time()
    fib(number)
    end = time.time()
    result = end - start
    print("Time fib normal: ",result,"s")

    start2 = time.time()
    func_lru_cached(number-1)
    end2 = time.time()
    result2 = end2 - start2
    print("Time fib cache: ",end2 - start2,"s")

    print(f'Cache is {round(result/result2,0)} times faster\n')


def test():
    print("For 10th element:")
    exec(10)

    print("For 20th element:")
    exec(20)

    print("For 30th element:")
    exec(30)

    print("For 35th element:")
    exec(35)
    
       
print("Test 1:")
print("-------")
test()
print("Test 2:")
print("-------")
test()
print("Test 3:")
print("-------")
test()
print("Test 4:")
print("-------")
test()
print("Test 5:")
print("-------")
test()


Test 1:
-------
For 10th element:
Time fib normal:  3.7670135498046875e-05 s
Time fib cache:  1.2636184692382812e-05 s
Cache is 3.0 times faster

For 20th element:
Time fib normal:  0.009050846099853516 s
Time fib cache:  1.4066696166992188e-05 s
Cache is 643.0 times faster

For 30th element:
Time fib normal:  0.6561970710754395 s
Time fib cache:  2.0503997802734375e-05 s
Cache is 32003.0 times faster

For 35th element:
Time fib normal:  6.900495529174805 s
Time fib cache:  1.049041748046875e-05 s
Cache is 657790.0 times faster

Test 2:
-------
For 10th element:
Time fib normal:  3.457069396972656e-05 s
Time fib cache:  1.9073486328125e-06 s
Cache is 18.0 times faster

For 20th element:
Time fib normal:  0.00479435920715332 s
Time fib cache:  2.86102294921875e-06 s
Cache is 1676.0 times faster

For 30th element:
Time fib normal:  0.6142387390136719 s
Time fib cache:  3.0994415283203125e-06 s
Cache is 198177.0 times faster

For 35th element:
Time fib normal:  7.485551834106445 s
Time fi

In [12]:
# zadanie 3

from sqlalchemy import create_engine, Column, String, Integer, Boolean
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base
from contextlib import contextmanager
from tempfile import mktemp
from platform import system


temp_db = mktemp(suffix='.sqlite')

print(f'Using {temp_db}')
print(f'--- Table: cities ---')
print(f'--- Columns: name, population, country  ---')

conn_uri_handler = {
    'Windows': f'sqlite:///{temp_db}',
    'Darwin': f'sqlite:////{temp_db}',
    'Linux': f'sqlite:////{temp_db}',
    'Java': f'sqlite:////{temp_db}'
}

engine = create_engine(conn_uri_handler[system()])
Base = declarative_base(bind=engine)

class City(Base):
    __tablename__ = 'cities'
    id=Column(Integer, primary_key=True,autoincrement=True)
    name=Column('name', String(50))
    population=Column('population', Integer)
    country=Column('country', String(50))

Base.metadata.create_all()
Session = sessionmaker(bind=engine)

@contextmanager
def create_session():
    session = Session()
    try:
        yield session
        session.commit()
    except Exception:
        session.rollback()
        raise
    finally:
        session.close()

def provide_session(func):
    def wrapper(*args, **kwargs):
        print(f'\n\tCalling {func.__name__}\n')
        try:
            with create_session() as session:
                args = (*args, session) if args else (session,) 
                return func(*args, **kwargs)
        except Exception as e:
            print(f'\tError found: {e}')

        return None

    return wrapper

@provide_session
def insert(session):
    name2 = input('\tEnter city name: ')
    
    population2 = input('\tEnter city population: ')
    if not population2.isnumeric():
        print('\t It to not numeric value!')
        while not population2.isnumeric():
            population2 = input('\tEnter city population: ')
            print('\t It to not numeric value!')
        
    country2 = input('\tEnter city country: ')
    
    session.add(City(name=name2, population=population2, country=country2))

@provide_session
def display(session):
    i = 0
    city = session.query(City).all()
    for cityN in city:
        i += 1
        print(f'\t{cityN.id} --- {cityN.name} --- {cityN.population} --- {cityN.country}')
        
    if i==0:
        print(f'\tNo records in db!')
        

@provide_session
def delete(session):
    x = input('\tRecord id: ')
    y = session.query(City).get(x)
    session.delete(y)

@provide_session   
def update(session):
    x = input('\tRecord id for update: ')
    z = input('\tColumn name for update: ')
    a = input('\tNew value: ')
    
    if z == 'name':
        session.query(City).filter(City.id == x).\
        update({City.name: a}, synchronize_session=False)
    elif z == 'population':
        session.query(City).filter(City.id == x).\
        update({City.population: a}, synchronize_session=False)
    elif z == 'country':
        session.query(City).filter(City.id == x).\
        update({City.country: a}, synchronize_session=False)

        
def displayMenu1():
    print(' ')
    print('1. Insert into table')
    print('2. Delete from table')
    print('3. Update table')
    print('4. Display table')
    print('5. End program')
        
def displayQuestion():
    p = True
    x = input('\tChoose one of the following option: ')
    if x=='1': insert()
    elif x=='2': delete()
    elif x=='3': update()
    elif x=='4': display()
    elif x=='5': p = False
    else:
        print('\n\tNot recognized option')
    print(' ')
    return p


# main program
while 1:        
    displayMenu1()
    if not displayQuestion():
        print('\tGoodbye!')
        break


Using /tmp/tmpht5cpk68.sqlite
--- Table: cities ---
--- Columns: name, population, country  ---
 
1. Insert into table
2. Delete from table
3. Update table
4. Display table
5. End program
	Choose one of the following option: 4

	Calling display

	No records in db!
 
 
1. Insert into table
2. Delete from table
3. Update table
4. Display table
5. End program
	Choose one of the following option: 1

	Calling insert

	Enter city name: Warsaw
	Enter city population: 2000000
	Enter city country: Poland
 
 
1. Insert into table
2. Delete from table
3. Update table
4. Display table
5. End program
	Choose one of the following option: 4

	Calling display

	1 --- Warsaw --- 2000000 --- Poland
 
 
1. Insert into table
2. Delete from table
3. Update table
4. Display table
5. End program
	Choose one of the following option: 1

	Calling insert

	Enter city name: New York
	Enter city population: 5000000
	Enter city country: USA
 
 
1. Insert into table
2. Delete from table
3. Update table
4. Display ta